In [29]:
#모듈 import
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas  as pd 
import psycopg2
from datetime import date, timedelta

In [30]:
# 자동 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(5)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url) values (%s, %s, %s);'
    return command

뷔스티에 원피스, 롱패딩, 숏패딩, 롱코트, 숏코트, 잠옷, 홈웨어, 트위드자켓, 블레이저 자켓, 싱글 자켓, 크롭 자켓, 레더 자켓, 루즈핏, 테일러드자켓, 린넨자켓, 반팔자켓, 라이더 자켓, 오버핏 자켓, 청자켓, 니트가디건, 볼레로, 아가일, 크롭가디건, 반팔가디건, 앙고라, 바람막이, 아노락, 윈드브레이커, 바시티자켓, 바시티점퍼, 야구점퍼, '여름아우터', '간절기아우터', '겨울아우터', '니트 맨투맨', '기모 맨투맨', '크롭 맨투맨', '남자 맨투맨', '반집업'

In [33]:
keyword_list = ['반팔 맨투맨', '크롭후드', '기모후드', '후드티', '남자 후드', '꾸안꾸', '플리스']

In [32]:
driver = webdriver.Chrome('C:\chromedriver.exe')

for query_txt in keyword_list :

    #크롬드라이버 실행
    driver.get('http://www.naver.com')
    time.sleep(2)

    #키워드를 검색한 네이버창 접속
    element = driver.find_element_by_id("query")
    element.send_keys(query_txt)
    element.submit()

    # 뷰 클릭
    driver.find_element_by_link_text("VIEW").click()
    time.sleep(2)

    #필터 클릭
    driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
    time.sleep(2)

    #기간 1년 설정
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()

    time.sleep(3)

    no2 = [ ]           # 게시글 번호 컬럼
    title2 = [ ]        # 게시물 제목 컬럼
    bdate2 = [ ]        # 작성 일자 컬럼
    url2 = [ ]          # 블로그 url 주소

    no = 1

    #더이상 스크롤이 안되면 종료
    i = 1
    time.sleep(1)

    #현재 스크롤 위치를 last_height로 받아옴
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        #스크롤 한 번 실행 후 스크롤 위치를 new_height로 받아옴
        scroll_down(driver)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")

        #더이상 스크롤이 안되면 종료
        if new_height == last_height:
            break

        last_height = new_height
        i += 1


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    view_list = soup.find('ul','lst_total').find_all('li')
    print(query_txt, '포스트 개수 :', len(view_list))

    for i in view_list :
        
        no2.append(no)                            # 게시물 번호 리스트에 추가
        print(no, end=' ')

        all_title = i.find_all('a')
        title = all_title[5].get_text( )          # 게시물 제목
        title2.append(title)                      # 게시물 제목 리스트에 추가

        time.sleep(2)

        bdate = i.find('span','sub_time sub_txt').get_text( )  # 작성일자
        bdate.replace('.', '-')
        bdate2.append(bdate)                     # 작성일자 리스트에 추가

        time.sleep(2)

        url = str(all_title[0])
        url = url.split('"')[5]                 # 게시물 url
        url2.append(url)                        # url 리스트에 추가

        time.sleep(2)

        if no == 200 : #한번에 200개 까지만 크롤링 가능
            break
            
        no += 1

    print(query_txt, '끝')

    bdate3 = [] #전처리 후 작성일자

    # 작성일자를 날짜형식으로 통일화
    for day in bdate2 :

        if '시간 전' in day :       # n시간 전 업로드 글
            day = date.today()
        elif '분 전' in day :       # n분 전 업로드 글
            day = date.today()
        elif '일 전' in day :       # n일 전 업로드 글 
            day = date.today() - timedelta(int(day[0])) #8일전 글 부터는 날짜로 반환
        elif day == '어제' :        # 어제 업로드 글
            day = date.today() - timedelta(1)
        else :
            day = day.rstrip('.')
            day = day.replace('.', '-')
        
        bdate3.append(day)

    # df화
    naver_blog = pd.DataFrame()
    naver_blog['제목'] = title2
    naver_blog['작성일자'] = bdate3
    naver_blog['주소'] = url2

    #date-time 타입으로 변환
    naver_blog['작성일자'] = pd.to_datetime(naver_blog['작성일자'])

    print(naver_blog.tail())

    #postgresql 접속
    conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    table = 'view_clothes'

    #table에 데이터 넣기
    for i in range(len(naver_blog)) :
        cur.execute(insert_command(table), (naver_blog.iloc[i][0], naver_blog.iloc[i][1], naver_blog.iloc[i][2]))
        conn.commit()

    conn.cursor().close()
    conn.close()

C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver.exe')
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:23: DeprecationWarnin

여름아우터 포스트 개수 : 107
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 여름아우터 끝
                                              제목       작성일자  \
102                        여아 옷 2세~ 4,5세 아우터 여름옷 2021-06-07   
103                                     여름 원피스 등 2022-03-27   
104                   여름 기본티/여름까지 쭈욱~경량점퍼등 구경하세요 2022-04-29   
105  단스킨 여자 여름 경량 아우터 (새상품) 등산하려고 1+1 사서 하나 팔아요! 2022-04-12   
106                                쟈딕 모닝 셔츠 XS/S 2022-04-27   

                                           주소  
102    https://cafe.naver.com/pajumom/3943701  
103  https://cafe.naver.com/jinheemom/2417637  
104      https://cafe.naver.com/mjann/3118851  
105    https://cafe.naver.com/pajumom/4120046  
106   https://cafe.naver.com/ttoks

C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


간절기아우터 포스트 개수 : 116
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 간절기아우터 끝
                                      제목       작성일자  \
111                     간절기 아우터 44~55사이즈 2022-03-12   
112        마시모두띠,바버,띠어리 등 간절기 아우터 새상품입니다 2022-03-30   
113                         모이몰른 간절기 아우터 2022-05-13   
114  빅사이즈,여성의류) 봄, 간절기 아우터,상의 (장당 3823원) 2022-02-24   
115                              간절기 아우터 2021-10-23   

                                          주소  
111    https://cafe.naver.com/yangmom/491265  
112     https://cafe.naver.com/mjann/3104334  
113  https://cafe.naver.com/imsanbu/60872048  
114      https://cafe.naver.com/soho/2921051  
115  https://cafe.naver.com/gloseems1/619241  


C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


...

남자 맨투맨 포스트 개수 : 89
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 남자 맨투맨 끝
                            제목       작성일자  \
84               남자 맨투맨 3장 새상품 2022-01-11   
85               메종키츠네 남성맨투맨 M 2022-01-22   
86  msgm 남성 맨투맨 사이즈 교환이나 판매합니다 2021-09-25   
87                       남자맨투맨 2022-02-09   
88          에르메스 22신상 남성맨투맨그린색 2022-02-08   

                                               주소  
84       https://cafe.naver.com/no1sejong/3203206  
85        https://cafe.naver.com/ttoksun2/3666861  
86        https://cafe.naver.com/kockoc11/2325857  
87  https://cafe.naver.com/chengnamomlife/1771620  
88    https://cafe.naver.com/parisienlook/3323440  


C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


반집업 포스트 개수 : 109
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 반집업 끝
                                    제목       작성일자  \
104                  나이키 반집업 판매합니다 3만원 2022-05-15   
105                파타고니아, 마무트, 몬츄라 반집업 2022-03-12   
106                   에이마켓 헤리티지반집업 구해요 2022-03-29   
107  일괄판매완료]나이키골프 반집업 바람막이,오클리스컬 반팔카라티 2022-05-24   
108                     폼스튜디오 세일러반집업세트 2022-04-19   

                                            주소  
104   https://cafe.naver.com/golmarket/9776711  
105  https://cafe.naver.com/chocammall/7404707  
106     https://cafe.naver.com/asadalin/502075  
107   https://cafe.naver.com/golmarket/9802991  
108      https://cafe.naver.com/gosaib/1015599  


C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_8928/4100996902.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()


KeyboardInterrupt: 

#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [17]:
#db 연결 종료
conn.cursor().close()
conn.close()